# Table of Contents

After [setting up](#Set-up) the python environment, we are going to:
- read the source file into memory
- determine the variable on which we will split files
- create an ordered list of values for that variable and save it as an object
- recursively do the following until we've done this for all chunks
    - overwrite the df with the slice that represents a chunk
    - add headers to the file
    - create a name for the file and save as an object
    - write the resulting file out to a folder with that name

# Set up

In [1]:
import pandas as pd
import numpy as np
import io, glob

# Initial Reading of Data (1st Lambda)

[back to top](#Table-of-Contents)

The data will flow through three folders:
- the input folder: 
    this folder holds the large files which need to be chunked
- the intermediate folder: 
    this folder will hold intermediate files which have data associated with an individual booklet.  The data in these files may not contain all events associated with a particular student, depending on the value assigned to the chunksize argument supplied to pandas.read_csv and the size of the data associate with that student 
- output folder:
    This is the folder where the final chunks will be placed. Each file here contains all of the events associated with the student indicated in the filename

In [5]:
####################################################
# This function 
# x = The assigned student number
# chunks = the number of individual chunks to assign
####################################################
def assignChunk(x,chunks=chunks):
    return min([c for c in chunks if x<c])

In [16]:

# fileListdfST17 = glob.glob("YangST4download/all_others/dfST1717*_book_*.csv")
# fileListdfST_17 = glob.glob("YangST4download/all_others/dfST_1717*_book_*.csv")
# fileListdfST7 = glob.glob("YangST4download/all_others/dfST7*_book_*.csv")

In [11]:
fileList = glob.glob('1717MA2N09CLIC30EX_M233.csv')
fileList

['1717MA2N09CLIC30EX_M233.csv']

The cell below creates the reference dataframe that deals with a single file.

In [2]:
df = pd.read_csv('C:/Users/cagard/Downloads/ST4/2017Math_1717MA4D01GXXX02EX_obs_tab_delimited_headers_CA.csv')

refdf = pd.DataFrame(df.BookletNumber.unique())
refdf = refdf.reset_index().rename(columns={'index':'Student', 0:'BookletNumber'})
print("There are %d unique Booklets in the source file."%refdf.BookletNumber.nunique())

There are 358228 unique Booklets in the source file.


The cell below deals with multiple input files:

In [4]:
####################################################
# Take all the files that exist in the combinedBook directory that start with 'dfST_' and ends with '.csv' and 
# build a list with those files and store them in the fileList variable
####################################################
# fileList = glob.glob("combinedBook/dfST_*.csv")
fileList = glob.glob("YangST4download/all_others/*1717MA2N09CLIC30EX*.csv")

####################################################
# make dataframe from the fileList list that was created
####################################################
refdf = pd.DataFrame(fileList)

####################################################
# reset the index to make the it start and then rename the column to 'Student' also
# the column that defaults to named '0' rename that to 'File'
####################################################
refdf = refdf.reset_index().rename(columns={'index':'Student', 0:'File'})
refdf['Student'] = refdf.Student
refdf

,Student,File
0,0,YangST4download/all_others/dfST_1717MA2N09CLIC...
1,1,YangST4download/all_others/dfST_1717MA2N09CLIC...
2,2,YangST4download/all_others/dfST_1717MA2N09CLIC...
3,3,YangST4download/all_others/dfST_1717MA2N09CLIC...
4,4,YangST4download/all_others/dfST_1717MA2N09CLIC...
...,...,...
28235,28235,YangST4download/all_others/dfST_1717MA2N09CLIC...
28236,28236,YangST4download/all_others/dfST_1717MA2N09CLIC...
28237,28237,YangST4download/all_others/dfST_1717MA2N09CLIC...
28238,28238,YangST4download/all_others/dfST_1717MA2N09CLIC...


# Chunking Data (also 1st Lambda)
Here we will try to save chunks. For each chunk in the file, we will:
- identify the list of booklets in that chunk and save all data for each booklet to it's own file.

Data will be dropped here first.

In [6]:
####################################################
#set number of booklets to a chunk  This is the same for either type of file
####################################################
chunksize = 1000
nchunks = int(np.ceil(refdf['Student'].max()/1000))
print('number of chunks',nchunks)

chunks = [(c+1) *chunksize for c in range(nchunks)]
chunks
####################################################
# take the 'Student' column and pass it to the function
# assignChunk() and assign the response to a new
# column in refdf.
####################################################
refdf['chunk'] = refdf['Student'].apply(assignChunk)

number of chunks 359


In [4]:
# This number will be off by one number, because this zero based index. 
refdf.Student.max()

358227

In [7]:
refdf.head()

,Student,BookletNumber,chunk
0,0,2020391905,1000
1,1,2210438182,1000
2,2,2140108786,1000
3,3,2863002272,1000
4,4,2093055816,1000


The below cell deals with a single file. The file name is not a factor for naming

In [22]:
####################################################
# Code improvement: creation of BlockCode column works for the current format
# must be improved for other formats (standard file format). 
# Ideally we should do this with regular expression
####################################################

# refdf['BlockCode'] = refdf.File[0].split('_')[-3].split('.csv')[0]
refdf['BlockCode'] = df.BlockCode[0]
for chunk in refdf.chunk.unique():
    try:
        refdf['startEnd'] = refdf.chunk.apply(lambda x: str(x-chunksize+1)+'-'+str(x))
#         print('refdf', refdf['startEnd'])
    #     refdf.startEnd.unique()
        #merge to file
#         dfLarge = dfLarge.merge(refdf,on='BookletNumber')
        refdf['fileType'] = "raw"
        refdf['outfile'] = refdf.fileType + "_" + refdf.BlockCode.astype(str)+'_'+refdf.startEnd.astype(str)+'.csv'
        refdf.head()
    except:
        print('exception')
        #write files to chunks
print('done')

done


In [23]:
refdf.head()

,Student,BookletNumber,chunk,BlockCode,startEnd,fileType,outfile
0,0,2333048613,1000,1717MA2N09CLIC30EX,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv
1,1,2333124506,1000,1717MA2N09CLIC30EX,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv
2,2,2333220343,1000,1717MA2N09CLIC30EX,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv
3,3,2333123640,1000,1717MA2N09CLIC30EX,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv
4,4,2333190959,1000,1717MA2N09CLIC30EX,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv


The below cell deals with multiple files with the file name included in the name of the file being read in

In [11]:
####################################################
# Code improvement: creation of BlockCode column works for the current format
# must be improved for other formats (standard file format). 
# Ideally we should do this with regular expression
####################################################

refdf['BlockCode'] = refdf.File[0].split('_')[-3].split('.csv')[0]
for chunk in refdf.chunk.unique():
    try:
        refdf['startEnd'] = refdf.chunk.apply(lambda x: str(x-chunksize+1)+'-'+str(x))
#         print('refdf', refdf['startEnd'])
    #     refdf.startEnd.unique()
        #merge to file
#         dfLarge = dfLarge.merge(refdf,on='BookletNumber')
        
        refdf['outfile'] = refdf.BlockCode.astype(str)+'_'+refdf.startEnd.astype(str)+'.csv'
        refdf.head()
    except:
        print('exception')
        #write files to chunks
print('done')

done


In [12]:
refdf.head()

,Student,File,chunk,BlockCode,startEnd,outfile
0,0,YangST4download/all_others/dfST_1717MA2N09CLIC...,1000,1717MA2N09CLIC30EX,1-1000,1717MA2N09CLIC30EX_1-1000.csv
1,1,YangST4download/all_others/dfST_1717MA2N09CLIC...,1000,1717MA2N09CLIC30EX,1-1000,1717MA2N09CLIC30EX_1-1000.csv
2,2,YangST4download/all_others/dfST_1717MA2N09CLIC...,1000,1717MA2N09CLIC30EX,1-1000,1717MA2N09CLIC30EX_1-1000.csv
3,3,YangST4download/all_others/dfST_1717MA2N09CLIC...,1000,1717MA2N09CLIC30EX,1-1000,1717MA2N09CLIC30EX_1-1000.csv
4,4,YangST4download/all_others/dfST_1717MA2N09CLIC...,1000,1717MA2N09CLIC30EX,1-1000,1717MA2N09CLIC30EX_1-1000.csv


Next cell needs to be modified to turn lambda function to an actual function with try except statements to print status of what is happening while running

In [ ]:
# Single file
print(df.shape)

df = df.merge(refdf, how="left", on=['BookletNumber', 'BlockCode'])
print(df.shape)
df.head()

In [26]:
# single file
df.groupby('outfile').apply(lambda x: x.to_csv("processedChunks233/"+x.outfile[0], index=False))

""


In [24]:
# multiple files
refdf.groupby('outfile').apply(lambda x: pd.concat([pd.read_csv(aFile) for aFile in x.File], sort=False)\
                               .to_csv("processedChunks233/"+x.outfile[0], index=False))

(1058554, 9)
(1058554, 14)


,EventId,Subject,BookletNumber,BlockCode,AccessionNumber,ItemTypeCode,Label,EventTime,ExtendedInfo,Student,chunk,startEnd,fileType,outfile
0,24300554,MAT,2333048613,1717MA2N09CLIC30EX,VH134373,FillInBlank,Resume Workstation,2017-01-31 16:29:07.840,NaN,0,1000,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv
1,732356571,MAT,2333124506,1717MA2N09CLIC30EX,VH134366,MultipleFillInBlank,Resume Workstation,2017-03-02 19:26:06.387,NaN,1,1000,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv
2,71213892,MAT,2333220343,1717MA2N09CLIC30EX,VH139196,CompositeCR,Resume Workstation,2017-02-01 14:30:51.887,NaN,2,1000,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv
3,815969460,MAT,2333123640,1717MA2N09CLIC30EX,VH098839,MCSS,Resume Workstation,2017-03-07 16:07:24.273,NaN,3,1000,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv
4,71213831,MAT,2333220343,1717MA2N09CLIC30EX,VH139196,CompositeCR,Resume Workstation,2017-02-01 14:23:44.167,NaN,2,1000,1-1000,raw,raw_1717MA2N09CLIC30EX_1-1000.csv


In [60]:
# Multiple file
refdf.groupby('outfile').apply(lambda x: pd.concat([pd.read_csv(aFile) for aFile in x.File], sort=False)\
                               .to_csv("processedChunks/"+x.outfile[0], index=False))

""


# Checking outfiles
We will read in a few of the files that were just generated from the code above. 
Checking the distribution of students.

In [70]:
lookdf = pd.read_csv('combinedBook/dfST_1717MA2N09CLIC30EX_1-1000.csv')
lookdf.head()
lookdf.columns

Index(['Unnamed: 0', 'AccNum', 'BlockCode', 'BookletNumber',
       'CalculatorBuffer', 'CalculatorModel', 'DialogueEvent',
       'DropChoiceResponse', 'EquationEditorButtonName',
       'EquationEditorInvokationMethod', 'EquationEditorKeyName',
       'EquationEditorStatus', 'ExtendedInfo', 'ItemOption',
       'ItemOptionStatus', 'ItemPart', 'ItemPartNumber', 'ItemSubPart',
       'ItemType', 'Label', 'Language', 'ObservableEventId', 'ProgressBar',
       'ScratchworkDrawMode', 'ScratchworkEraseMode',
       'ScratchworkHighlightMode', 'ScratchworkOverlay', 'ScrollArea',
       'ScrollDirection', 'ScrollMethod', 'ScrollPixelsX', 'ScrollPixelsY',
       'SubjectCode', 'TTS', 'TTSRead', 'TextBoxNumber', 'contentLaTeX',
       'contentMathML', 'numericIdentifier', 'partId', 't', 'theme',
       'zoomLevel', 'VH098519_0-0.0', 'VH098522_0-0.0', 'VH098556_0-0.0',
       'VH098597_0-0.0', 'VH098740_0-0.0', 'VH098753_0-0.0', 'VH098759_0-0.0',
       'VH098779_0-0.0', 'VH098783_0-0.0', 'VH09

In [27]:
qcdf = pd.read_csv('processedChunks233/raw_1717MA2N09CLIC30EX_1-1000.csv')
qcdf.columns

Index(['EventId', 'Subject', 'BookletNumber', 'BlockCode', 'AccessionNumber',
       'ItemTypeCode', 'Label', 'EventTime', 'ExtendedInfo', 'Student',
       'chunk', 'startEnd', 'fileType', 'outfile'],
      dtype='object')

In [28]:
outFileList = glob.glob("processedChunks233/raw_1717*.csv")
print(len(outFileList))

3


In [29]:
####################################################
# Note: No more than one file can have fewer number of Students than the chunk size
#     No files should have more than the number of chunks. 
#     All files should have the same number of columns.
# This will show how many students as well as the shape (columns)
####################################################
# df = pd.read_csv(outFileList[0])
# print(df.BookletNumber.nunique(), df.shape[1])
# df.head()
for file in outFileList:
    tempdf = pd.read_csv(file)
    print('File {}: Number of Students (Booklet Numbers) {}\n Number of columns {} (columns names: {})\n'.format(file, tempdf.BookletNumber.nunique(), tempdf.shape[1], tempdf.columns))
    

File processedChunks233/raw_1717MA2N09CLIC30EX_1001-2000.csv: Number of Students (Booklet Numbers) 1000
 Number of columns 14 (columns names: Index(['EventId', 'Subject', 'BookletNumber', 'BlockCode', 'AccessionNumber',
       'ItemTypeCode', 'Label', 'EventTime', 'ExtendedInfo', 'Student',
       'chunk', 'startEnd', 'fileType', 'outfile'],
      dtype='object'))

File processedChunks233/raw_1717MA2N09CLIC30EX_2001-3000.csv: Number of Students (Booklet Numbers) 882
 Number of columns 14 (columns names: Index(['EventId', 'Subject', 'BookletNumber', 'BlockCode', 'AccessionNumber',
       'ItemTypeCode', 'Label', 'EventTime', 'ExtendedInfo', 'Student',
       'chunk', 'startEnd', 'fileType', 'outfile'],
      dtype='object'))

File processedChunks233/raw_1717MA2N09CLIC30EX_1-1000.csv: Number of Students (Booklet Numbers) 1000
 Number of columns 14 (columns names: Index(['EventId', 'Subject', 'BookletNumber', 'BlockCode', 'AccessionNumber',
       'ItemTypeCode', 'Label', 'EventTime', 'Ex

In [45]:
refdf.loc[refdf.chunk == 29000].File.nunique()

236

To do: 

figure out a way to QC logs for files that were created

need to investigate the issue with the difference in the number of columns in various files (some have 73, 74, 75, 76 etc. columns)

In [69]:
# writefiles = {{k:v} for zip(k,v) in zip(Booklets.startEnd.unique())}
# def writefile(x,filecol='outfile', index=False):
#     filename=x[filecol]
#     if x[filecol].unique()==1:
#         x.to_csv(filename,index=index)
for file in dfLarge.outfile.unique():
    print('Processing {}...'.format(file))
    dfLarge.loc[dfLarge.outfile==file].to_csv(file,index=False)
    print('{} successsfully processed!'.format(file))